In [ ]:
import os
import zipfile
import subprocess
import shutil
from google.colab import files

def install_dependencies():
    """Menginstall library yang diperlukan."""
    print("🔄 Menginstall library 'kaggle' dan 'split-folders'...")
    subprocess.check_call(['pip', 'install', '-q', 'kaggle', 'split-folders'])
    print("✅ Library berhasil diinstall.")

def setup_kaggle_api():
    """Menangani upload token kaggle.json dan konfigurasi."""
    print("\n📂 Silakan upload file 'kaggle.json' Anda sekarang:")

    uploaded = files.upload()

    if 'kaggle.json' not in uploaded:
        print("❌ Error: File 'kaggle.json' tidak ditemukan. Harap upload file yang benar.")
        return False

    print("🔐 Mengkonfigurasi API Token...")
    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

    os.chmod('/root/.kaggle/kaggle.json', 0o600)
    print("✅ Konfigurasi Kaggle API selesai.")
    return True

def download_and_unzip():
    """Mendownload dataset ASL Alphabet dan mengekstraknya."""
    dataset_name = "grassknoted/asl-alphabet"
    zip_file = "asl-alphabet.zip"

    print(f"\n⬇️ Sedang mendownload dataset '{dataset_name}' dari Kaggle...")
    subprocess.run(['kaggle', 'datasets', 'download', '-d', dataset_name], check=True)

    if os.path.exists(zip_file):
        print("📦 Sedang mengekstrak dataset (ini mungkin memakan waktu beberapa saat)...")
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall("raw_data")
        print("✅ Ekstraksi selesai.")

        os.remove(zip_file)
        return True
    else:
        print("❌ Gagal mendownload dataset.")
        return False

def split_dataset():
    """Membagi dataset menjadi Train, Validation, dan Test."""
    import splitfolders

    input_folder = "raw_data/asl_alphabet_train/asl_alphabet_train"
    output_folder = "asl_split_data"

    if not os.path.exists(input_folder):
        print(f"❌ Folder input tidak ditemukan di: {input_folder}")
        print("   Mengecek struktur folder 'raw_data'...")
        print(os.listdir("raw_data"))
        return

    print(f"\n✂️ Membagi dataset menjadi: Train (80%), Val (10%), Test (10%)...")
    print("   Proses ini akan menyalin dan mengacak file gambar, mohon tunggu...")

    splitfolders.ratio(
        input_folder,
        output=output_folder,
        seed=1337,
        ratio=(.8, .1, .1), # 80% Train, 10% Val, 10% Test
        group_prefix=None
    )

    print(f"✅ Selesai! Dataset siap digunakan di folder: '{output_folder}'")
    print(f"   Struktur folder: {os.listdir(output_folder)}")

def main():
    try:
        install_dependencies()
        if setup_kaggle_api():
            if download_and_unzip():
                split_dataset()
                print("\n🎉 Semua proses berhasil! Anda siap untuk training model.")
    except Exception as e:
        print(f"\n❌ Terjadi kesalahan: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import tensorflow as tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def create_data_generators():
    base_dir = 'asl_split_data'
    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')
    test_dir = os.path.join(base_dir, 'test')

    IMG_HEIGHT = 224  # Standar MobileNetV2
    IMG_WIDTH = 224
    BATCH_SIZE = 32

    print("🔄 Mengkonfigurasi ImageDataGenerator...")

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    print("✅ Generator dikonfigurasi.")
    print("📂 Memuat data dari direktori...")

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical', # Multiclass classification
        shuffle=True
    )

    validation_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, validation_generator, test_generator

if __name__ == "__main__":

    if os.path.exists('asl_split_data'):
        train_gen, val_gen, test_gen = create_data_generators()

        print(f"\n🏷️ Kelas ditemukan ({len(train_gen.class_indices)}):")
        labels = list(train_gen.class_indices.keys())
        print(labels[:10], "... dan lainnya.")

        print("\n✅ Siap untuk training dengan MobileNetV2!")
    else:
        print("❌ Error: Folder 'asl_split_data' tidak ditemukan.")
        print("   Jalankan script 'prepare_asl_dataset.py' terlebih dahulu.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(num_classes=29, input_shape=(224, 224, 3)):
    """
    Membangun model Transfer Learning menggunakan MobileNetV2.

    Args:
        num_classes (int): Jumlah kelas output (ASL Alphabet biasanya 29).
        input_shape (tuple): Dimensi gambar input (224, 224, 3).

    Returns:
        model: Model Keras yang sudah dikompilasi.
    """

    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = False
    x = base_model.output

    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation='relu')(x)

    x = Dropout(0.5)(x)

    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    optimizer = Adam(learning_rate=0.0001)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

if __name__ == "__main__":
    model = build_model(num_classes=29)

    print("✅ Model MobileNetV2 berhasil dibangun dan dikompilasi.")
    model.summary()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from google.colab import files
try:
    from preprocess_asl_data import create_data_generators
    from model_asl_mobilenet import build_model
except ImportError:
    print("⚠️ Error: Pastikan file 'preprocess_asl_data.py' dan 'model_asl_mobilenet.py' sudah ada.")

def plot_training_history(history):
    """
    Membuat visualisasi grafik Akurasi dan Loss.
    """
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    # Plot Akurasi
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.grid(True)

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.grid(True)

    plt.show()

def run_training():
    print("Memulai persiapan data...")
    train_gen, val_gen, test_gen = create_data_generators()

    num_classes = train_gen.num_classes
    print(f"Terdeteksi {num_classes} kelas.")

    print("Membangun model MobileNetV2...")
    model = build_model(num_classes=num_classes)

    checkpoint_path = "best_asl_model.keras"

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]

    print("Mulai proses training selama 20 Epoch...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        callbacks=callbacks,
        verbose=1
    )

    print("\n Mengevaluasi model pada Test Set...")
    test_loss, test_acc = model.evaluate(test_gen)
    print(f"   Test Accuracy: {test_acc*100:.2f}%")
    print(f"   Test Loss: {test_loss:.4f}")

    print("\nMenampilkan grafik hasil training...")
    plot_training_history(history)

    print(f"\nMengunduh file model '{checkpoint_path}' ke komputer lokal...")
    if os.path.exists(checkpoint_path):
        files.download(checkpoint_path)
    else:
        print("File model tidak ditemukan untuk didownload.")

if __name__ == "__main__":
    run_training()